Steps are:

- select a reconstruction time
- the code determines which paleogeography stage this falls within, gets the start and end times
- load the relevant precomputed multipoint files, and in the process assign an integer to the different types for use in interpolation steps (e.g. set land to be 1, shallow marine to be 2, etc)

- for land and marine



In [6]:
import sys
sys.path.append('../paleogeography')
sys.path.append('../paleotopography')
sys.path.append('../pigplates/')
sys.path.append('../pygplates2014/')

import pygplates
import glob, re
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import xarray as xr

import polygon_processing as pp
import paleogeography as pg
import paleogeography_tweening as pgt
import paleotopography as pt

from proximity_query import *
from create_gpml import create_gpml_regular_long_lat_mesh
import points_in_polygons
from sphere_tools import sampleOnSphere
import points_spatial_tree



%matplotlib inline
%load_ext autoreload
%autoreload 2


reconstruction_basedir = '../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles/'
tween_basedir = './tween_feature_collections_gpml/'
file_format = 'gpml'

output_dir = './paleotopo_grids_netcdf4/'

netcdf3_output = False

#rotation_model = pygplates.RotationModel(['%s/Global_EB_250-0Ma_GK07_Matthews++.rot' % reconstruction_basedir,
#                                          '%s/Global_EB_410-250Ma_GK07_Matthews++.rot' % reconstruction_basedir])
rotation_model = pygplates.RotationModel('%s/Global_EarthByte_230-0Ma_GK07_AREPS.rot' % reconstruction_basedir)


COBterrane_file = '%s/Global_EarthByte_GeeK07_COB_Terranes_Matthews_etal.gpml' % reconstruction_basedir

agegrid_file_template = '/Users/Simon/Data/AgeGrids/Agegrids_30m_20151002_2015_v1_r756/agegrid_30m_%d.grd'


#############################################
## Set the heights for different environment
#############################################
depth_for_unknown_ocean = -1000
# ----------------------------------
shallow_marine_elevation = -200.
# ----------------------------------
lowland_elevation = 200.
# ----------------------------------
max_mountain_elevation = 1500.
# NOTE - this height is actually the mountain height IN ADDITION TO the lowland height
# so that the maximum absolute elevation would be [lowland_elevation + max_mountain_elevation]
# TODO should call this 'mountain_relief'???
#############################################

# the grid sampling for the output
sampling = 0.5

# this number controls how small polygons are exclude when merging the COB terranes into 
# land/sea masking polygons
area_threshold = 0.0001

# used for quadtree
subdivision_depth = 2

# this buffer defines the smoothness of the topography at the transition from 'lowland' to 'mountain'
# the distance defined here is the distance over which heights ramp from the lowland elevation to the 
# mountain elevation defined above. (the ramping takes place from the edge of the mountain range inwards
# towards the mountain interior). Any parts of the mountain range greater than this buffer distance from 
# the edge will have a uniform height equal to max_mountain_elevation
mountain_buffer_distance_degrees = 1.
#mountain_buffer_distance_degrees = 2.

# choose here either 'ocean' or 'land'
# this determines which grid takes precedence where both the age grid and the 
# paleogeographies overlap and contain valid values
land_or_ocean_precedence = 'land'

# this number is used in the final grdfilter step to smooth the output 
grid_smoothing_wavelength_kms = 400.

time_min = 0.
time_max = 230.
time_step = 1.

merge_with_bathymetry = True


####################################################

# make a sorted list of the (midpoint) times for paleogeography polygons
tmp = glob.glob('%s/*/' % reconstruction_basedir)

time_list = []
for tm in tmp:
    time_list.append(float(re.findall(r'\d+Ma+',tm)[1][:-2]))

time_list.sort()

time_list = np.array(time_list)

print time_list


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[   6.   14.   22.   33.   45.   53.   76.   90.  105.  126.  140.  152.
  169.  195.  218.  232.  255.  277.  287.  302.  328.  348.  368.  396.]


In [8]:

time_list = np.hstack((0,time_list))


def paleotopography_job()

    print 'Working on Time %0.2fMa\n' % reconstruction_time 
        
    # find times that bracket the selected exact time in the paleogeography source files
    time_stage_max = time_list[np.where(time_list>reconstruction_time)[0][0]]
    time_stage_min = time_list[np.where(time_list<=reconstruction_time)[0][-1]]

    # Note the logic for selecting the times:
    # The main issue is that each set of paleogeography polygons has a defined 'midpoint' time
    # --> if the reconstruction time is between these, the choice of t1 and t2 is obvious
    # --> if the reconstruction time matches one of these times, then we can work directly on
    #     the geometries that match this time - hence the two routes through the if statement below
    
    print 'Selected Time is in the stage %0.2fMa to %0.2fMa' % (time_stage_min,time_stage_max)

    land_points_file = '%s/tweentest_land_%0.2fMa_%0.2fMa.%s' % (tween_basedir,time_stage_min,time_stage_max,file_format)
    marine_points_file = '%s/tweentest_ocean_%0.2fMa_%0.2fMa.%s' % (tween_basedir,time_stage_min,time_stage_max,file_format)
    mountains_going_up_file = '%s/mountain_transgression_%0.2fMa_%0.2fMa.%s' % (tween_basedir,time_stage_min,time_stage_max,file_format)
    mountains_going_down_file = '%s/mountain_regression_%0.2fMa_%0.2fMa.%s' % (tween_basedir,time_stage_min,time_stage_max,file_format)
    mountains_stable_file = '%s/mountain_stable_%0.2fMa_%0.2fMa.%s' % (tween_basedir,time_stage_min,time_stage_max,file_format)

    
    # get a nx3 array defining the points above sea-level, reconstructed to time of interest
    # columns are [lat, long, elevation assigned for lowland]
    land_point_array = pt.add_reconstructed_points_to_xyz(land_points_file,
                                                          rotation_model,
                                                          reconstruction_time,
                                                          lowland_elevation)
    
    # get a nx3 array defining shallow marine areas, reconstructed to time of interest
    # columns are [lat, long, elevation assigned for shallow marine]
    marine_point_array = pt.add_reconstructed_points_to_xyz(marine_points_file,
                                                            rotation_model,
                                                            reconstruction_time,
                                                            shallow_marine_elevation)
    
    # Note that the two arrays just created are based on 'regular' lat/long grids, but 
    # are not aligned with the regular lat/long grid that we want to output
    # since they are (usually) reconstructed to a different time from the one at which they
    # were created (and anyway may be at a different resolution to the grid sampling specified
    # here)

    # combine the previous two arrays
    pg_point_array = np.vstack((land_point_array,marine_point_array))

    # get a merged version of COB terranes, optionally excluding polygons that are small in area
    # TODO deal with donut polygons better
    sieve_polygons = pt.get_merged_cob_terrane_polygons(COBterrane_file,rotation_model,
                                                        reconstruction_time,sampling)

    # get arrays defining the land and sea based on which points fall within the COB terranes
    # NOTE this step is where we create the points that ARE on the regular lat/long grid we 
    # will ultimately output
    (lat,lon,zval,
     lat_deep,lon_deep,zval_deep) = pt.get_land_sea_multipoints(sieve_polygons,sampling,depth_for_unknown_ocean,
                                                                subdivision_depth=subdivision_depth)


    # sample the land/marine points onto the points within the COB Terranes
    # This will fill the gaps that exist within continents, and average out overlaps
    d,l = sampleOnSphere(pg_point_array[:,0],pg_point_array[:,1],pg_point_array[:,2],
                         np.array(lat),np.array(lon),n=1)

    land_marine_interp_points = pg_point_array[:,2].ravel()[l]

    # At this point, the land points are all considered to be 'lowland'......
    
    ####################################
    # Deal with the mountains
    if np.equal(reconstruction_time,time_stage_min):
        print 'Temporary fix for valid time'
        #dat3 = add_reconstructed_points_to_xyz(mountains_going_up_file,rotation_model,reconstruction_time,3)
        dat4 = pt.add_reconstructed_points_to_xyz(mountains_going_down_file,rotation_model,reconstruction_time+0.01,1)
        dat5 = pt.add_reconstructed_points_to_xyz(mountains_stable_file,rotation_model,reconstruction_time+0.01,1)
        mountains_tr_point_array = np.vstack((dat4,dat5))
        
        dist_tr = pt.get_distance_to_mountain_edge(mountains_tr_point_array,reconstruction_basedir,
                                                   rotation_model,reconstruction_time,area_threshold)
        dist_tr_cap = np.array(dist_tr)
        dist_tr_cap[np.array(dist_tr)>mountain_buffer_distance_degrees] = mountain_buffer_distance_degrees

        dist_tr_cap = dist_tr_cap*mountains_tr_point_array[:,2]

        normalized_mountain_elevation = dist_tr_cap
        
    else:
        # load in the mountain points but at three different times: t1 and t2, and the reconstruction time
        # note that these three arrays should all be identical in size, since they are the same multipoints
        # just reconstructed to three slightly different times
        dat3 = pt.add_reconstructed_points_to_xyz(mountains_going_up_file,rotation_model,time_stage_max,1)
        dat4 = pt.add_reconstructed_points_to_xyz(mountains_going_down_file,rotation_model,time_stage_max,1)
        dat5 = pt.add_reconstructed_points_to_xyz(mountains_stable_file,rotation_model,time_stage_max,1)
        mountains_t2_point_array = np.vstack((dat3,dat4,dat5))

        dat3 = pt.add_reconstructed_points_to_xyz(mountains_going_up_file,rotation_model,time_stage_min+0.01,1)
        dat4 = pt.add_reconstructed_points_to_xyz(mountains_going_down_file,rotation_model,time_stage_min+0.01,1)
        dat5 = pt.add_reconstructed_points_to_xyz(mountains_stable_file,rotation_model,time_stage_min+0.01,1)
        mountains_t1_point_array = np.vstack((dat3,dat4,dat5))

        dat3 = pt.add_reconstructed_points_to_xyz(mountains_going_up_file,rotation_model,reconstruction_time,1)
        dat4 = pt.add_reconstructed_points_to_xyz(mountains_going_down_file,rotation_model,reconstruction_time,1)
        dat5 = pt.add_reconstructed_points_to_xyz(mountains_stable_file,rotation_model,reconstruction_time,1)
        mountains_tr_point_array = np.vstack((dat3,dat4,dat5))

        # calculate distances of the mountain points to the edge of the mountain region at t1 and t2,
        # using the pg polygons that they should exactly correspond to 
        dist_t1 = pt.get_distance_to_mountain_edge(mountains_t1_point_array,reconstruction_basedir,
                                                   rotation_model,time_stage_min,area_threshold)
        dist_t2 = pt.get_distance_to_mountain_edge(mountains_t2_point_array,reconstruction_basedir,
                                                   rotation_model,time_stage_max,area_threshold)
        
        #is_in_orogeny_index = find_mountain_type(mountains_tr_point_array,
        #                                         orogeny_feature_filename,
        #                                         reconstruction_time)


        # cap the distances at some arbitrary value defined earlier
        dist_t1_cap = np.array(dist_t1)
        dist_t1_cap[np.array(dist_t1)>mountain_buffer_distance_degrees] = mountain_buffer_distance_degrees

        dist_t1_cap = dist_t1_cap*mountains_t1_point_array[:,2]
        
        dist_t2_cap = np.array(dist_t2)
        dist_t2_cap[np.array(dist_t2)>mountain_buffer_distance_degrees] = mountain_buffer_distance_degrees
        
        dist_t2_cap = dist_t2_cap*mountains_t2_point_array[:,2]

        # get the normalised time within this time stage
        # for example we are at 0.25 between the t1 and t2
        t_diff = (time_stage_max-time_stage_min)
        t_norm = (reconstruction_time-time_stage_min)/t_diff
        #print t_diff, t_norm

        # use 1d interpolation to get the 'normalized' height of the mountains at the preceding
        # and subsequent times to the specific reconstruction time
        # [note this is not spatial interpolation - rather it is interpolation at each individual point
        # between the heights at earlier and later times]
        tmp = np.vstack((dist_t1_cap,dist_t2_cap))
        f = interpolate.interp1d([0,1],tmp.T)
        normalized_mountain_elevation = f(t_norm)
    
    
    #plt.figure(figsize=(25,11))
    #plt.plot(mountains_tr_point_array[:,1],mountains_tr_point_array[:,0],'.')
    
    # interpolate the elevations at tr onto the regular long lat points that we will ultimately use 
    # for the grid output
    # note the k value here controls number of neighbouring points used in inverse distance average
    d,l = sampleOnSphere(mountains_tr_point_array[:,0],mountains_tr_point_array[:,1],normalized_mountain_elevation,
                         np.array(lat),np.array(lon),k=4)
    w = 1./d**2
    normalized_mountain_elevation_interp_points = np.sum(w * normalized_mountain_elevation.ravel()[l],axis=1) / np.sum(w,axis=1)

    # this index isolates only those points that are within a certain distance of the mountain range
    # (since the interpolation will give values everywhere in the 'land', so we want to re-isolate only 
    # those points that fall within the 'mountain' regions)
    # TODO this should be set to the sampling??
    mountain_proximity_index = np.degrees(np.min(d,axis=1))< sampling*2 #mountain_buffer_distance_degrees

    plotting=False
    if plotting:
        plt.figure(figsize=(25,11))
        plt.scatter(mountains_tr_point_array[:,1],mountains_tr_point_array[:,0],
                    c=normalized_mountain_elevation,
                    edgecolor='',s=2)
        plt.colorbar()

        plt.figure(figsize=(25,11))
        plt.scatter(np.array(lon)[mountain_proximity_index],
                    np.array(lat)[mountain_proximity_index],
                    c=normalized_mountain_elevation_interp_points[mountain_proximity_index],
                    edgecolor='',s=2)
        plt.colorbar()
        
        plt.figure(figsize=(25,11))
        plt.scatter(mountains_tr_point_array[:,1],mountains_tr_point_array[:,0],
                    c=mountains_t1_point_array[:,2],
                    edgecolor='',s=4)
        plt.colorbar()

        plt.figure(figsize=(25,11))
        plt.scatter(np.array(lon),
                    np.array(lat),
                    c=normalized_mountain_elevation_interp_points,
                    edgecolor='',s=2)
        plt.colorbar()

        plt.figure(figsize=(25,11))
        plt.scatter(np.array(lon),
                    np.array(lat),
                    c=np.min(d,axis=1),
                    edgecolor='',s=2)
        plt.colorbar()

    
    #####################################
    # Put the grid together
    #####################################
    
    # write the land/marine points to a file
    pt.write_xyz_file('tmp/land_marine.xyz',zip(lon+lon_deep,
                                            lat+lat_deep,
                                            np.hstack((land_marine_interp_points,zval_deep))))

    # convert the normalized mountain elevations to metres, then write to file
    mountain_elevation_factor = max_mountain_elevation/mountain_buffer_distance_degrees
    mountain_elevation_array = normalized_mountain_elevation_interp_points[mountain_proximity_index]*mountain_elevation_factor
    pt.write_xyz_file('tmp/mountain.xyz',zip(np.array(lon)[mountain_proximity_index],
                                         np.array(lat)[mountain_proximity_index],
                                         mountain_elevation_array))

    # all the points are already on the same regular lat/long grid (but with gaps) - just 
    # need to piece them all together and combine.
    # Note we assume the the mountain elevation is the height IN ADDITION to the lowland elevation
    # so that we can simply add them
    os.system('gmt xyz2grd tmp/land_marine.xyz -Gtmp/land_marine.nc -Rd -I%0.8f' % sampling)
    os.system('gmt xyz2grd tmp/mountain.xyz -Gtmp/mountain.nc -Rd -I%0.8f -di0' % sampling)
    os.system('gmt grdmath tmp/mountain.nc tmp/land_marine.nc ADD = %s/paleotopo_%0.2fMa.nc' % (output_dir,reconstruction_time))

    # load result back into python
    topoX,topoY,topoZ = pg.load_netcdf('%s/paleotopo_%0.2fMa.nc' % (output_dir,reconstruction_time))

    
    if merge_with_bathymetry:
    
        # PALEOBATHYMETRY based on age grids
        # load age grid for this time and calculate paleobathymetry
        agegrid_file = agegrid_file_template % reconstruction_time

        ageX,ageY,ageZ = pg.load_netcdf(agegrid_file)

        paleodepth = pg.age2depth(ageZ,model='GDH1')


        # get index for grid nodes where age grid is nan, replace values with topography/shallow bathymetry
        not_bathy_index = np.isnan(paleodepth)
        paleodepth[not_bathy_index] = topoZ[not_bathy_index]


        # save the merged grid (forcing compatibility with GPlates-readable netCDF in case it helps)
        ds = xr.DataArray(paleodepth,
                          coords=[('lat',topoY),('lon',topoX)],
                          name='elevation')
        ds.to_netcdf('tmp/paleotopobathy.nc',format='NETCDF3_CLASSIC')

        # smooth the grid using GMT [wavelength is optional
        #pg.smooth_topography_grid('paleotopobathy.nc','paleotopobathy_smooth_%0.2fMa.nc' % reconstruction_time,400.)
        os.system('gmt grdfilter %s -G%s -Fg%0.2f -fg -D4 -Vl' % ('tmp/paleotopobathy.nc',
                                                                 'tmp/paleotopobathy_smooth.nc',
                                                                 grid_smoothing_wavelength_kms))

        if netcdf3_output:
            # finally, once again force GPlates-readable netCDF (ie netCDF v3) and put the 
            # grid in the output folder with a filename containing the age
            os.system('gmt grdconvert %s -G%s=cf' % ('tmp/paleotopobathy_smooth.nc',
                                                     '%s/paleotopobathy_smooth_%0.2fMa.nc' % (output_dir,reconstruction_time)))
        else:
            os.system('cp tmp/paleotopobathy_smooth.nc %s/paleotopobathy_smooth_%0.2fMa.nc' % (output_dir,reconstruction_time))
        
        # load and plot the result
        topo_smoothX,topo_smoothY,topo_smoothZ = pg.load_netcdf('tmp/paleotopobathy_smooth.nc')
        #
        plt.figure(figsize=(25,11))
        plt.imshow(topo_smoothZ,origin='lower',
                   extent=[-180,180,-90,90],
                   cmap=plt.cm.terrain,vmin=-5000,vmax=5000)
        plt.title('%0.2fMa' % reconstruction_time)
        plt.colorbar()
        plt.savefig('%s/paleotopobathy_smooth_%0.2fMa.png' % (output_dir,reconstruction_time))
        plt.close()

        

for reconstruction_time in np.arange(time_min,time_max+time_step,time_step):


Working on Time 0.00Ma

Selected Time is in the stage 0.00Ma to 6.00Ma
Temporary fix for valid time
./present_day_paleogeography.gmt


/Users/Simon/GIT/paleogeography/paleogeography.py:351: RuntimeWarning: invalid value encountered in greater_equal
  paleodepth[age_array>=20.] = 5651 - 2473*np.exp(-0.0278*age_array[age_array>=20.])


Working on Time 1.00Ma

Selected Time is in the stage 0.00Ma to 6.00Ma
./present_day_paleogeography.gmt
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_6Ma/m_fig64_11_2_PresentDay_Paleogeog_Matthews2016_6.00Ma.shp']
Working on Time 2.00Ma

Selected Time is in the stage 0.00Ma to 6.00Ma
./present_day_paleogeography.gmt
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_6Ma/m_fig64_11_2_PresentDay_Paleogeog_Matthews2016_6.00Ma.shp']
Working on Time 3.00Ma

Selected Time is in the stage 0.00Ma to 6.00Ma
./present_day_paleogeography.gmt
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_6Ma/m_fig64_11_2_PresentDay_Paleogeog_Matthews2016_6.00Ma.shp']
Working on Time 4.00Ma

Selected Time is in the stage 0.00Ma to 6.00Ma
./present_day_paleogeography.gmt
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleoge

KeyboardInterrupt: 